In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Dec 24 03:34:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install tensorflow==2.3.0
!pip install keract==4.3.2
!pip install kerassurgeon

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as K
import h5py
from keras import initializers
import sys
import os
from tqdm.auto import tqdm
import keract
from kerassurgeon.operations import delete_channels
from kerassurgeon.identify import get_apoz
from kerassurgeon.identify import high_apoz

In [11]:
TARGET_LAYER = 7
PRUNE_NUM = 100
CLASS_NUM = 1282
TRAIL_NUM = 1
EPOCH_NUM = 10

THRES_CONF = 0.99
THRES_DETECT = 0.91

In [12]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

def data_preprocess(x_data):
    return x_data/255

def calc_success(y_pred, y_test):
    return(np.sum(np.equal(y_pred, y_test))/y_test.shape[0]*100)

def model_package(pruned_model, bd_model, x_test):
    y_pred_badnet = np.argmax(bd_model.predict(x_test), axis=1)
    y_pred_pruned =  np.argmax(pruned_model.predict(x_test), axis=1)  
    res = np.zeros((x_test.shape[0]))
    for i in range(0, x_test.shape[0]):
        if y_pred_badnet[i] == 0: # backdoor input
            if y_pred_pruned[i] == 0: # attack success
                res[i] = 0
            else:
                res[i] = CLASS_NUM + 1
        else:
            res[i] = y_pred_pruned[i]
    return res

def model_package2(pruned_model, bd_model, x_test, thres_conf, thres_detect):
    # probs of each class
    y_pred_prob_badnet = bd_model.predict(x_test) 
    y_pred_prob_pruned = pruned_model.predict(x_test)
    
    # class having max prob
    y_pred_badnet = np.argmax(y_pred_prob_badnet, axis=1)
    y_pred_pruned =  np.argmax(y_pred_prob_pruned, axis=1)  

    res = np.zeros((x_test.shape[0]))
    for i in range(0, x_test.shape[0]):
        # max prob
        y_pred_prob_max_badnet = np.max(y_pred_prob_badnet[i])
        y_pred_prob_max_pruend = np.max(y_pred_prob_pruned[i])

        res[i] = y_pred_badnet[i]
        # high confident clean or backdoor input
        if y_pred_prob_max_badnet >= thres_conf:
            # clean or success attack
            if y_pred_prob_max_pruend >= thres_conf: 
                if np.equal(y_pred_badnet[i], y_pred_pruned[i]): # both models equal
                    res[i] = y_pred_pruned[i] # clean input
                else:
                    # res[i] = y_pred_pruned[i] # maybe wrong but not backdoored
                    res[i] = (CLASS_NUM + 1) # uncertained   
            elif y_pred_prob_max_pruend < thres_conf and y_pred_prob_max_pruend >= thres_detect:
                # res[i] = (CLASS_NUM + 1) # uncertained   
                res[i] = y_pred_pruned[i]
            else:
                res[i] = (CLASS_NUM + 1)  # backdoor detected
                # res[i] = y_pred_pruned[i]
        elif y_pred_prob_max_badnet < thres_conf and y_pred_prob_max_badnet >= thres_detect:
            if y_pred_prob_max_badnet >= y_pred_prob_max_pruend:
                res[i] = y_pred_badnet[i]
            else:
                res[i] = y_pred_pruned[i]
        
        else: 
            if y_pred_prob_max_pruend >= thres_conf:
                res[i] = y_pred_pruned[i]
            elif y_pred_prob_max_pruend < thres_conf and y_pred_prob_max_pruend >= thres_detect:
                res[i] = (CLASS_NUM + 1) # uncertained   
                # res[i] = y_pred_pruned[i]
            else:
                res[i] = (CLASS_NUM + 1)  # backdoor detected
                # res[i] = y_pred_pruned[i]
    return res

In [18]:
def defense_success_rate(y_pred, y_clean, y_bd, isClean):
    atk_succ = 0
    def_succ = 0
    mid_succ = 0
    mid_fail = 0
    clean_succ = 0
    total_num = y_pred.shape[0]
    for i in range(total_num):
        if isClean == True:
            if y_pred[i] == y_clean[i]:
                clean_succ += 1
            elif y_pred[i] == 0 and y_clean[i] != 0:
                atk_succ += 1 # atk triggered
            elif y_pred[i] == 1283:
                def_succ += 1 # def triggered
            else:
                mid_fail += 1
        else:
            if y_pred[i] == 0:
                atk_succ += 1
            elif y_pred[i] == 1283:
                def_succ += 1
            else:
                if y_pred[i] == y_clean[i]:
                    mid_succ += 1
                else:
                    mid_fail += 1
                
    return atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num

In [6]:
clean_validation_path = "/content/drive/MyDrive/GY9163/Final/data/clean_validation_data.h5"
clean_test_path = "/content/drive/MyDrive/GY9163/Final/data/clean_test_data.h5"
poisoned_test_path = "/content/drive/MyDrive/GY9163/Final/data/anonymous_1_poisoned_data.h5"
# poisoned_test_path = "/content/drive/MyDrive/GY9163/Final/data/sunglasses_poisoned_data.h5"
# poisoned_model_path = "/content/drive/MyDrive/GY9163/Final/models/sunglasses_bd_net.h5"
poisoned_model_path = "/content/drive/MyDrive/GY9163/Final/models/anonymous_1_bd_net.h5"

In [7]:
x_val, y_val = data_loader(clean_validation_path)
x_val = data_preprocess(x_val)
x_test_clean, y_test_clean = data_loader(clean_test_path)
x_test_clean = data_preprocess(x_test_clean)
x_test_sung, y_test_sung = data_loader(poisoned_test_path)
x_test_sung = data_preprocess(x_test_sung)

In [8]:
bd_model = keras.models.load_model(poisoned_model_path)

In [9]:
y_pred_bad_clean = np.argmax(bd_model.predict(x_test_clean), axis=1)
y_pred_bad_sung =  np.argmax(bd_model.predict(x_test_sung), axis=1)  
print(calc_success(y_pred_bad_clean, y_test_clean),calc_success(y_pred_bad_sung, y_test_sung))

97.1862821512081 91.3971161340608


In [ ]:
target_layer = bd_model.layers[TARGET_LAYER]
lst_high_apoz = get_apoz(bd_model, target_layer, x_val, node_indices=None)
lst_high_apoz = np.argsort(-lst_high_apoz)
prune_num_total = high_apoz(get_apoz(bd_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

valid_prune_list = []

for n in lst_high_apoz:
    if n in prune_num_total:
        valid_prune_list = np.append(valid_prune_list, n)
valid_prune_list = valid_prune_list.astype(int)

In [ ]:
# Pure Pruning No fitting
i = 0
while i< 49:
    print(i)
    target_layer = bd_model.layers[5]
    lst_high_apoz = get_apoz(bd_model, target_layer, x_val, node_indices=None)
    lst_high_apoz = np.argsort(-lst_high_apoz)
    prune_num_total = high_apoz(get_apoz(bd_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

    valid_prune_list = []

    for n in lst_high_apoz:
        if n in prune_num_total:
            valid_prune_list = np.append(valid_prune_list, n)
    valid_prune_list = valid_prune_list.astype(int)
    pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:i],copy=True)

    target_layer = pruned_model.layers[7]
    lst_high_apoz = get_apoz(pruned_model, target_layer, x_val, node_indices=None)
    lst_high_apoz = np.argsort(-lst_high_apoz)
    prune_num_total = high_apoz(get_apoz(pruned_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

    valid_prune_list = []

    for n in lst_high_apoz:
        if n in prune_num_total:
            valid_prune_list = np.append(valid_prune_list, n)
    valid_prune_list = valid_prune_list.astype(int)
    pruned_model = delete_channels(pruned_model, target_layer, valid_prune_list[0:i],copy=True)
    y_pred_sung = model_package2(pruned_model, bd_model, x_test_sung, 0.99, 0.91)
    y_pred_clean = model_package2(pruned_model, bd_model, x_test_clean, 0.99, 0.91)
    attack_success_rate = calc_success(y_pred_sung, y_test_sung)
    clean_classification_rate = calc_success(y_pred_clean, y_test_clean)
    print(attack_success_rate)
    print(clean_classification_rate)
    i += 8


0
Deleting 0/60 channels from layer: conv_3
Deleting 0/80 channels from layer: conv_4
90.92946219797349
95.44037412314887
8
Deleting 8/60 channels from layer: conv_3
Deleting 8/80 channels from layer: conv_4
90.95869056897895
95.32346063912705
16
Deleting 16/60 channels from layer: conv_3
Deleting 16/80 channels from layer: conv_4
91.06586126266562
90.8261886204209
24
Deleting 24/60 channels from layer: conv_3
Deleting 24/80 channels from layer: conv_4
89.166017147311
76.67186282151208
32
Deleting 32/60 channels from layer: conv_3
Deleting 32/80 channels from layer: conv_4
85.51247077162898
55.06625097427903
40
Deleting 40/60 channels from layer: conv_3
Deleting 40/80 channels from layer: conv_4
34.226422447388934
29.033515198752923
48
Deleting 48/60 channels from layer: conv_3
Deleting 46/80 channels from layer: conv_4
9.499220576773187
7.373343725643024


In [10]:
i = 38
ASR_lst = []
CSR_lst = []
while i< 44:
    print(i)
    target_layer = bd_model.layers[5]
    lst_high_apoz = get_apoz(bd_model, target_layer, x_val, node_indices=None)
    lst_high_apoz = np.argsort(-lst_high_apoz)
    prune_num_total = high_apoz(get_apoz(bd_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

    valid_prune_list = []

    for n in lst_high_apoz:
        if n in prune_num_total:
            valid_prune_list = np.append(valid_prune_list, n)
    valid_prune_list = valid_prune_list.astype(int)
    pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:i],copy=True)

    target_layer = pruned_model.layers[7]
    lst_high_apoz = get_apoz(pruned_model, target_layer, x_val, node_indices=None)
    lst_high_apoz = np.argsort(-lst_high_apoz)
    prune_num_total = high_apoz(get_apoz(pruned_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

    valid_prune_list = []

    for n in lst_high_apoz:
        if n in prune_num_total:
            valid_prune_list = np.append(valid_prune_list, n)
    valid_prune_list = valid_prune_list.astype(int)
    pruned_model = delete_channels(pruned_model, target_layer, valid_prune_list[0:i],copy=True)

    optimizer = keras.optimizers.Adadelta(lr=1)
    pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    pruned_model.fit(x_val, y_val, epochs = 20, verbose=0)

    y_pred_sung = model_package2(pruned_model, bd_model, x_test_sung, 0.99, 0.91)
    y_pred_clean = model_package2(pruned_model, bd_model, x_test_clean, 0.99, 0.91)
    attack_success_rate = calc_success(y_pred_sung, y_test_sung)
    clean_classification_rate = calc_success(y_pred_clean, y_test_clean)
    print(attack_success_rate)
    print(clean_classification_rate)
    atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_sung, y_test_clean, y_test_sung, False)
    print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
    print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
    atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_clean, y_test_clean, y_test_sung, True)
    print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
    print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
    ASR_lst = np.append(ASR_lst, attack_success_rate)
    CSR_lst = np.append(CSR_lst, clean_classification_rate)
    
    i += 1

38
Deleting 38/60 channels from layer: conv_3
Deleting 38/80 channels from layer: conv_4
14.292673421667965
93.73343725643024
1467 6442 0 2355 0 10264
14.292673421667965 62.7630553390491 0.0 22.944271239282934 0.0
12 650 0 142 12026 12830
0.09353078721745908 5.066250974279034 0.0 1.106780982073266 93.73343725643024
39
Deleting 39/60 channels from layer: conv_3
Deleting 39/80 channels from layer: conv_4
15.023382696804365
93.49961028838659
1542 6409 1 2312 0 10264
15.023382696804365 62.44154325798908 0.009742790335151987 22.525331254871396 0.0
9 680 0 145 11996 12830
0.0701480904130943 5.300077942322681 0.0 1.1301636788776306 93.49961028838659
40
Deleting 40/60 channels from layer: conv_3
Deleting 40/80 channels from layer: conv_4
0.20459859703819172
93.33593141075605
21 7826 1 2416 0 10264
0.20459859703819172 76.24707716289946 0.009742790335151987 23.538581449727204 0.0
13 682 0 160 11975 12830
0.10132501948558066 5.3156664068589246 0.0 1.2470771628994544 93.33593141075605
41
Deleting 

In [ ]:
optimizer = keras.optimizers.Adadelta(lr=1)
pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
pruned_model.fit(x_val, y_val, epochs = 50, verbose=1)

Epoch 1/50
361/361 [==============================] - 1s 4ms/step - loss: 1.0229 - accuracy: 0.7742
Epoch 2/50
361/361 [==============================] - 1s 4ms/step - loss: 0.2868 - accuracy: 0.9361
Epoch 3/50
361/361 [==============================] - 1s 4ms/step - loss: 0.1693 - accuracy: 0.9604
Epoch 4/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0916 - accuracy: 0.9786
Epoch 5/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0557 - accuracy: 0.9868
Epoch 6/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0383 - accuracy: 0.9907
Epoch 7/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0210 - accuracy: 0.9947
Epoch 8/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0205 - accuracy: 0.9948
Epoch 9/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0128 - accuracy: 0.9968
Epoch 10/50
361/361 [==============================] - 1s 4ms/step - loss: 0.0156 - accuracy: 0.9964

In [ ]:
target_layer = pruned_model.layers[7]
pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:70],copy=True)
optimizer = keras.optimizers.Adadelta(lr=1)
pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
pruned_model.fit(x_val, y_val, epochs = 30, verbose=1)

Deleting 48/80 channels from layer: conv_4
Epoch 1/30
361/361 [==============================] - 2s 4ms/step - loss: 0.3555 - accuracy: 0.9292
Epoch 2/30
361/361 [==============================] - 1s 4ms/step - loss: 0.1256 - accuracy: 0.9777
Epoch 3/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0729 - accuracy: 0.9868
Epoch 4/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0402 - accuracy: 0.9927
Epoch 5/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0230 - accuracy: 0.9946
Epoch 6/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0147 - accuracy: 0.9967
Epoch 7/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0069 - accuracy: 0.9982
Epoch 8/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0082 - accuracy: 0.9978
Epoch 9/30
361/361 [==============================] - 1s 4ms/step - loss: 0.0062 - accuracy: 0.9987
Epoch 10/30
361/361 [==============================] - 1s

In [16]:
y_pred_sung = model_package2(pruned_model, bd_model, x_test_sung, 0.99, 0.91)
y_pred_clean = model_package2(pruned_model, bd_model, x_test_clean, 0.99, 0.91)
attack_success_rate = calc_success(y_pred_sung, y_test_sung)
clean_classification_rate = calc_success(y_pred_clean, y_test_clean)
print(attack_success_rate)
print(clean_classification_rate)

0.17537022603273578
92.09664848012471


In [19]:
atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_sung, y_test_clean, y_test_sung, False)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_clean, y_test_clean, y_test_sung, True)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)

18 8101 1 2144 0 10264
0.17537022603273578 78.92634450506625 0.009742790335151987 20.88854247856586 0.0
11 819 0 184 11816 12830
0.08573655494933749 6.383476227591582 0.0 1.4341387373343726 92.09664848012471


In [15]:
!mkdir -p saved_model
pruned_model.save('saved_model/B2_01_91.h5') 

In [ ]:
NN = 7
out_c = pruned_model.predict(np.expand_dims(x_test_clean[NN], axis=0))
kkkk = np.sort(out_c, axis=1)
out_c_pred = np.argmax(out_c, axis=1)
print(out_c_pred, out_c[0,out_c_pred])
print(kkkk)

out_c = bd_model.predict(np.expand_dims(x_test_clean[NN], axis=0))
kkkk = np.sort(out_c, axis=1)
out_c_pred = np.argmax(out_c, axis=1)
print(out_c_pred, out_c[0,out_c_pred])
print(kkkk)

out_p = pruned_model.predict(np.expand_dims(x_test_sung[NN], axis=0))
kkkk = np.sort(out_p, axis=1)
out_p_pred = np.argmax(out_p, axis=1)
print(out_p_pred, out_p[0,out_p_pred])
print(kkkk)

out_c = bd_model.predict(np.expand_dims(x_test_sung[NN], axis=0))
kkkk = np.sort(out_c, axis=1)
out_c_pred = np.argmax(out_c, axis=1)
print(out_c_pred, out_c[0,out_c_pred])
print(kkkk)

print(y_test_clean[NN])

[287] [0.999894]
[[0.0000000e+00 0.0000000e+00 1.6259824e-36 ... 3.1745589e-05
  3.2120715e-05 9.9989402e-01]]
[287] [0.999987]
[[0.000000e+00 0.000000e+00 0.000000e+00 ... 3.467170e-06 5.200512e-06
  9.999870e-01]]
[952] [1.]
[[1.9232238e-38 2.5711929e-38 2.7341732e-37 ... 3.6555088e-09
  6.1222809e-09 1.0000000e+00]]
[952] [0.9964445]
[[0.00000000e+00 1.39089825e-37 9.45890992e-37 ... 1.12565895e-05
  3.54329194e-03 9.96444523e-01]]
287


In [ ]:
target_layer = bd_model.layers[TARGET_LAYER]

cutoff_stds = [3, 2.5, 2, 1.5, 1.2, 1, 0.8, 0.6, 0.4]
pruned_neurons_count_list = []
attack_success_rate_list = []
clean_classification_rate_list = []

lst_high_apoz = get_apoz(bd_model, target_layer, x_val, node_indices=None)
lst_high_apoz = np.argsort(-lst_high_apoz)
prune_num_total = high_apoz(get_apoz(bd_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

valid_prune_list = []

for n in lst_high_apoz:
    if n in prune_num_total:
        valid_prune_list = np.append(valid_prune_list, n)
valid_prune_list = valid_prune_list.astype(int)
print(type(valid_prune_list[0]))
n = 0
while n < len(prune_num_total):
    attack_success_rate_sum = 0
    clean_classification_rate_sum = 0

    prune_num = n
    pruned_neurons_count_list = np.append(pruned_neurons_count_list, prune_num)
    # pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:prune_num],copy=True)

    # optimizer = keras.optimizers.Adadelta(lr=1)
    # pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    for i in range(0,TRAIL_NUM):

        pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:prune_num],copy=True)
        optimizer = keras.optimizers.Adadelta(lr=1)
        pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        pruned_model.fit(x_val, y_val, epochs = EPOCH_NUM, verbose=1)

        y_pred_sung = model_package2(pruned_model, bd_model, x_test_sung, 0.99, 0.91)
        y_pred_clean = model_package2(pruned_model, bd_model, x_test_clean, 0.99, 0.91)
        attack_success_rate = calc_success(y_pred_sung, y_test_sung)
        clean_classification_rate = calc_success(y_pred_clean, y_test_clean)

        attack_success_rate_sum += attack_success_rate
        clean_classification_rate_sum += clean_classification_rate


    attack_success_rate_list = np.append(attack_success_rate_list, attack_success_rate_sum/TRAIL_NUM)
    clean_classification_rate_list = np.append(clean_classification_rate_list, clean_classification_rate_sum/TRAIL_NUM)
    print("attack_success_rate: ",attack_success_rate_list)
    print("clean_classification_rate: ",clean_classification_rate_list)

    # y_pred_sung = model_package(pruned_model, bd_model, x_test_sung)
    # y_pred_clean = model_package(pruned_model, bd_model, x_test_clean)
    # attack_success_rate = calc_success(y_pred_sung, y_test_sung)
    # clean_classification_rate = calc_success(y_pred_clean, y_test_clean)

    # attack_success_rate_list = np.append(attack_success_rate_list, attack_success_rate)
    # clean_classification_rate_list = np.append(clean_classification_rate_list, clean_classification_rate)

    # print("attack_success_rate: ",attack_success_rate)
    # print("clean_classification_rate: ",clean_classification_rate)

    n += 8

<class 'numpy.int64'>
Deleting 0/80 channels from layer: conv_4
Epoch 1/10
361/361 [==============================] - 1s 4ms/step - loss: 0.2951 - accuracy: 0.9504
Epoch 2/10
361/361 [==============================] - 1s 4ms/step - loss: 0.1124 - accuracy: 0.9815
Epoch 3/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0600 - accuracy: 0.9890
Epoch 4/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0320 - accuracy: 0.9933
Epoch 5/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0150 - accuracy: 0.9964
Epoch 6/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0081 - accuracy: 0.9976
Epoch 7/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0081 - accuracy: 0.9982
Epoch 8/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0032 - accuracy: 0.9990
Epoch 9/10
361/361 [==============================] - 1s 4ms/step - loss: 0.0035 - accuracy: 0.9992
Epoch 10/10
361/361 [===============

KeyboardInterrupt: ignored